# Data Exploration

This notebook focuses on the exploration and modeling of 📄 [ERA5 Dataset – DOI: 10.24381/cds.f17050d7](https://doi.org/10.24381/cds.f17050d7) provided by the **Copernicus CDS**.

We will work with **ERA5 monthly averaged reanalysis data** (1981–2010), specifically the **skin temperature** variable.  
The goal is to explore, process and analyze monthly skin temperature anomalies from the ERA5 dataset.

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import polars as pl
import xarray as xr
import numpy as np

### Load and inspect the ERA5 dataset (GRIB format)

**Dataset summary highlights**  
- **Size & coverage**: ~1 GB of data on a 0.25° × 0.25° grid, spanning 360 months (1981-01 through 2010-12).  
- **Coordinates**:  
    - `time`: monthly timestamps (360)  
    - `latitude`: 90.0 → -90.0 (721 points)  
    - `longitude`: 0.0 → 359.75 (1440 points)  
- **Data variable**:  
    - `skt` in **Kelvin** (`units: K`, `long_name: Skin temperature`)  
    - Stored as `float32`, with missing values encoded as NaN  
- **Metadata & conventions**:  
    - Follows **CF-1.7** conventions, produced by ECMWF (GRIB_edition 1)  
    - Attributes such as `institution`, `history` and GRIB grid details document provenance and grid definition  

- **Auxiliary coords** (`number`, `step`, `surface`, `valid_time`): GRIB-specific metadata that can be dropped for core analysis. 

In [ ]:
grib_path = "../reanalysis-era5-single-levels-monthly-means.grib"
ds = xr.open_dataset(grib_path, engine="cfgrib", decode_timedelta=False)
print(ds)

The GRIB attributes confirm the field is on a regular lat-lon grid with 0.25° spacing. No missing data except the standard GRIB sentinel (`≈3.4e38`) already mapped to NaN by cfgrib.

In [ ]:
skt = ds["skt"]
print("\nVariable summary:")
print(skt)
print("\nVariable attributes:")
for k, v in skt.attrs.items():
    print(f"  {k}: {v}")

### Inspecting the Skin Temperature Field

**Unit conversion**
- We convert the raw skin temperature field from **Kelvin to °C** and removed the singleton GRIB coords (`number`, `step`, `surface`) for a clean 3D array.  
- The computed range spans **−76.93 °C** (extreme polar winter values) up to **+46.06 °C** (intense summer warmth over deserts), which seems sensible

In [ ]:
# Convert to °C and drop singleton coords
skt_C = (skt - 273.15).squeeze(drop=True)
t_min = float(skt_C.min())
t_max = float(skt_C.max())

print(f"Skin-temperature range: {t_min:.2f} °C  -> {t_max:.2f} °C")

**Global descriptive statistics**  

Here, `skt_C.values.ravel()` pulls out the entire 3D array of skin‐temperature values (time x lat x long) and flattens it into one continuous 1D array so that global statistics can be computed over all points in a single step.

Some hints: 
- **Global mean (5.60 °C)** is lower than the **median (10.74 °C)**, indicating a distribution skewed toward colder extremes.  
- **1st percentile (−56.55 °C)** and **99th percentile (32.67 °C)** capture the most extreme events (e.g. polar winters, desert heat waves).  

In [ ]:
val = skt_C.values.ravel()  # 360 x 721 x 1440 ~ 370M points
stats = {
    "mean": float(np.nanmean(val)),
    "median": float(np.nanmedian(val)),
    "p01": float(np.nanpercentile(val, 1)),
    "p25": float(np.nanpercentile(val, 25)),
    "p75": float(np.nanpercentile(val, 75)),
    "p99": float(np.nanpercentile(val, 99)),
    "min": float(np.nanmin(val)),
    "max": float(np.nanmax(val)),
}
print("Global statistics for 1981-2010")
for k, v in stats.items():
    print(f"{k:>6}: {v:6.2f} °C")

**Histogram insight:**  

The global distribution of monthly skin temperature values shows a pronounced peak around **25–30 °C** (tropical and desert regions) and a secondary plateau near **0 °C** (mid latitude oceans). A long cold tail extends down to **−76 °C** (polar winters) confirming strong skewness toward lower temperatures. 

In [ ]:
counts, edges = np.histogram(val, bins=40)

centers = 0.5 * (edges[:-1] + edges[1:])
plt.figure(figsize=(7, 5))
plt.bar(centers, counts, width=np.diff(edges), color="steelblue", alpha=0.9)
plt.title("Distribution of monthly skin-temperature (°C)")
plt.xlabel("°C")
plt.ylabel("Count")
plt.tight_layout()

### Inspect a few spatial snapshots

Now that we know the absolute range of the data, we will:

1. **Inspect a few spatial snapshots** (e.g. January vs. July) to appreciate geographic contrast.
2. **Check the global mean time series** to confirm there is no unwanted trend.

**Seasonal snapshots of skin temperature**  

- **January 1990** (top): strong polar cooling in the Northern Hemisphere, with large areas below 0 °C. Mid‐latitude landmasses still show moderate winter temperatures, while tropical oceans remain above 20 °C.  
- **July 1990** (bottom): peak summer warmth in the Northern Hemisphere, especially over the Sahara, Middle East and central Asia (> 40 °C). Southern high latitudes now display their cold season (< 0 °C) and equatorial regions stay relatively uniform (~25 °C).  


These two maps illustrate the opposite phases of the annual cycle and **motivate the removal of the seasonal mean** when computing anomalies.  

In [ ]:
for sel_date in ["1990-01-01", "1990-07-01"]:
    fig = plt.figure(figsize=(9, 4))
    ax = plt.axes(projection=ccrs.PlateCarree())

    skt_C.sel(time=sel_date).squeeze().plot.pcolormesh(
        ax=ax,
        transform=ccrs.PlateCarree(),
        cmap="coolwarm",
        vmin=-60,
        vmax=45,
        add_colorbar=True,
    )
    ax.coastlines(color="black", linewidth=0.6)
    ax.set_title(f"Skin temperature (°C) - {sel_date[:7]}")
    plt.tight_layout()


**Spatial variability (std) of monthly skin temperature 1981–2010**  

This map shows the standard deviation of skin temperature at each grid cell over 360 months.  

Here, `std_map = skt_C.std("time")` computes, for each grid cell (fixed latitude & longitude), the **standard deviation** of its 360 monthly skin‐temperature values.  
Mathematically, at each (lat, lon) point it calculates  
$$
\sigma = \sqrt{\frac{1}{N-1}\sum_{t=1}^{N}\bigl(T_t - \bar{T}\bigr)^2},
$$  
where $T_t$ are the monthly temperatures from January 1981 to December 2010 and $\bar{T}$ is their mean.  

The resulting `std_map` highlights how strongly each location’s temperature fluctuates through the annual cycle and interannual variability.
- **High σ (yellow)** in polar and continental regions indicates large seasonal swings (e.g. Arctic winters vs. summers, desert extremes).  
- **Low σ (purple/blue)** over tropical oceans reflects minimal month to month variation due to the buffering effect of water.  

Understanding this spatial pattern helps identify regions where anomalies will be most pronounced once the regular seasonal cycle is removed.  

In [ ]:
std_map = skt_C.std("time")

plt.figure(figsize=(9, 4))
std_map.plot.pcolormesh(cmap="viridis", vmin=0, vmax=10, add_colorbar=True)
plt.title("Std (monthly skin-T, 1981-2010)  [°C]")
plt.tight_layout()

### Removing the Seasonal Cycle and Computing Monthly Anomalies

**Monthly climatology (`clim`)**  
- Dimensions: **12 months** × 721 latitudes** × 1440 longitudes, corresponding to **each calendar month’s average over 1981–2010**.  
- Size: ~50 MB of gridded data on the native 0.25° × 0.25° ERA5 grid.  
- Values: mean skin temperature in °C for each month (e.g. `clim.sel(month=1)` is the January climatology).  
- Coordinates:  
  - `month`: 1–12  
  - `latitude`: 90.0 → -90.0  
  - `longitude`: 0.0 → 359.75  
  
 This `clim` array provides the baseline seasonal cycle to subtract from the raw data and compute anomalies.  

In [ ]:
clim = skt_C.groupby("time.month").mean("time")
print(clim)

**Anomaly DataArray (`anom`)**  

- Dimensions: **360 time steps** × **721 latitudes** × **1440 longitudes** (same grid as original), containing monthly anomalies for 1981–2010.  
- Values: deviations in °C from each month’s 30 year mean (negative values indicate colder-than-usual months; positive values indicate warmer than usual months).  
 
This `anom` array is now centered on zero for each calendar month, isolating the irregular (interannual) variability and ready for further analysis.  

In [ ]:
anom = skt_C.groupby("time.month") - clim
print(anom)

**Global skin‐temperature anomaly**  

This map shows the deviation of surface skin‐temperature from the 1981–2010 August climatology, plotted on a Plate Carree projection with true coastlines and gridlines for geographic reference. Positive (red) and negative (blue) anomalies highlight regions warmer or cooler than usual that month.  
 
**August 1997 was the peak of a record breaking El Niño event** and the anomaly pattern reflects its classic footprint:  
- The pronounced warm anomaly along the equatorial Pacific (around 120° W–90° W) aligns with the Niño 3.4 region, where skin‐temperature rose by +3–4 °C.  
- Cooling (blue) over Indonesia and northern Australia reflects suppressed convection typical of El Niño.  
- Widespread positive anomalies across subtropical oceans and continents mirror the global warm phase of this event.  

In [ ]:
fig = plt.figure(figsize=(10, 4))
ax = plt.axes(projection=ccrs.PlateCarree())

anom.sel(time="1997-08").squeeze().plot.pcolormesh(
    ax=ax,
    transform=ccrs.PlateCarree(),
    cmap="RdBu_r",
    vmin=-5,
    vmax=5,
    add_colorbar=True,
)
ax.coastlines(resolution="110m", color="black", linewidth=1.0)
ax.set_title("Skin temperature anomaly (°C) – Aug 1997", pad=12)

gl = ax.gridlines(draw_labels=True, linewidth=0.3, color="gray", alpha=0.5)
gl.top_labels = False
gl.right_labels = False

plt.tight_layout()
plt.show()

**Histogram of skin temperature anomalies**  

- We flatten the entire `anom` array (`anom.values.ravel()`) to a 1D vector of 370 million anomaly values (space x time).  
- A NumPy histogram with 40 bins between –5 °C and +5 °C captures the full anomaly range, focusing on the most common deviations.  
- The resulting bar chart shows a roughly Gaussian distribution centered at zero, with long tails out to ±5 °C confirming that most anomalies are small (±1–2 °C) while extreme deviations are rare.  

In [ ]:
anom_vals = anom.values.ravel()
counts, edges = np.histogram(anom_vals, bins=40, range=(-5, 5))
centers = 0.5 * (edges[:-1] + edges[1:])

plt.figure(figsize=(7, 5))
plt.bar(centers, counts, width=np.diff(edges), color="tomato", alpha=0.8)
plt.title("Distribution of skin-temperature anomalies (°C)")
plt.xlabel("Anomaly (°C)")
plt.ylabel("Count")
plt.tight_layout()
plt.show()


**All values anomaly statistics (space x time)**  

- **Minimum anomaly: −24.28 °C** - the coldest departure from the monthly norm anywhere, likely a polar winter outlier.  
- **Median: ≈ 0 °C** — half of all anomalies are above and half below zero, confirming symmetry around the climatology.  
- **Maximum anomaly: +18.21 °C** — the most extreme warm departure, e.g. a localized heat wave relative to the seasonal mean.  
  
> These statistics demonstrate that while most anomalies fall within ±1 °C, the tails extend out to ±5 °C for 1 % of cases, with rare extremes beyond ±18 °C at specific locations and times.  

In [ ]:
anom_vals = anom.values.ravel()

stats_all = {
    "min": float(np.nanmin(anom_vals)),
    "p01": float(np.nanpercentile(anom_vals, 1)),
    "p25": float(np.nanpercentile(anom_vals, 25)),
    "median": float(np.nanpercentile(anom_vals, 50)),
    "p75": float(np.nanpercentile(anom_vals, 75)),
    "p99": float(np.nanpercentile(anom_vals, 99)),
    "max": float(np.nanmax(anom_vals)),
}

print("All values anomaly stats (°C) – 1981-2010")
for k, v in stats_all.items():
    print(f"{k:>6}: {v:+.2f}")

This time series shows the **global mean anomaly**, meaning that for each month we **averaged** the anomaly values over **every latitude and longitude grid cell**. It represents the planet wide departure from the 1981–2010 monthly climatology.  
Averages the anomaly field over all latitudes and longitudes, collapsing the 3D array into a single number (the global mean anomaly).

Key features we can observe:

- Peaks around late 1997 early 1998, corresponding to the strong El Niño event.
- Troughs during La Niña phases (e.g. 1984–85, 1999–2000).
- A slight upward drift over the three decades, reflecting the impact of global warming on the global mean anomaly.

This series highlights both interannual variability (ENSO cycles) and any longer term trends in global skin‐temperature departures from the seasonal norm.


In [ ]:
plt.figure(figsize=(12, 4))
(anom.mean(dim=["latitude", "longitude"])).plot()
plt.title("Global mean skin‐temperature anomaly (°C) – 1981–2010")
plt.ylabel("Anomaly (°C)")
plt.xlabel("Time")
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

**Global mean anomaly statistics averaged over the entire globe (1981–2010)**  
  
These values show that global‐mean anomalies remain small (within ±1 °C), reflecting how regional extremes are dampened when averaged planet wide.  

In [ ]:
gma = anom.mean(dim=["latitude", "longitude"])

stats = {
    "min": float(gma.min()),
    "p01": float(gma.quantile(0.01)),
    "p25": float(gma.quantile(0.25)),
    "median": float(gma.quantile(0.50)),
    "p75": float(gma.quantile(0.75)),
    "p99": float(gma.quantile(0.99)),
    "max": float(gma.max()),
}

print("Global mean anomaly (°C) - 1981-2010 statistics")
for k, v in stats.items():
    print(f"{k:>6}: {v:+.2f}")